<img src="../common/rfsoc_book_banner.jpg" alt="University of Strathclyde" align="left">

<div class="alert alert-block" style="background-color: #c7b8d6; padding: 10px">
    <p style="color: #222222">
        <b>Note:</b>
        <br>
        This Jupyter notebook uses hardware features of the Zynq UltraScale+ RFSoC device. Therefore, the notebook cells will only execute successfully on an RFSoC platform.
    </p>
</div>

# Notebook Set G

---

## 03 - RFSoC Radio Hello World!
The final notebook in this set will use the RFSoC radio system to send messages from the transmitter to the receiver. The first message we will send is 'Hello World!'. Then, we will use a timer thread class to send a repeating message. A text terminal widget will be used to inspect received messages from the transmitter. Lastly, an image will be sent from the transmitter to the receiver. Received images will be displayed in the Jupyter notebook.

## Table of Contents
* [1. Introduction](#introduction)
* [2. Receiving Messages](#receiving-messages)
* [3. Hello World!](#hello-world)
* [4. A Repeating Message](#a-repeating-message)
* [5. Image Transfer](#image-transfer)
* [6. Conclusion](#conclusion)

## References
* [1] - [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.4, November 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_4/pg269-rf-data-converter.pdf)
* [2] - [Stewart, R. W., Barlee, K. W., Atkinson, D. S. W., & Crockett, L. H. (2015). Software Defined Radio using MATLAB & Simulink and the RTL-SDR. (1 ed.)](https://www.desktopsdr.com/)

## Revision
* **v1.0** | 13/01/23 | *First Revision*

---


## 1. Introduction
Let us begin by programming the FPGA bitstream and initialising the PYNQ overlay design. To do this, we need to import the `rfsoc_radio` package.

Upon executing the code cell below, two tests will run to ensure the radio system is operational. If these tests fail, check your platform's loopback connection, restart the kernel, and try running the code cell again.

<div class="alert alert-box alert-danger">
<b>Caution:</b>
    In this demonstration, we generate signals using the RFSoC development board. Your device should be setup in loopback mode. You should understand that the RFSoC platform can also transmit RF signals wirelessly. Remember that unlicensed wireless transmission of RF signals may be illegal in your geographical location. Radio signals may also interfere with nearby devices, such as pacemakers and emergency radio equipment. Note that it is also illegal to intercept and decode particular RF signals. If you are unsure, please seek professional support.
</div>

In [ ]:
from rfsoc_radio.overlay import RadioOverlay

ol = RadioOverlay(run_test=True)

In the previous notebook, we launched a dashboard to control the radio system. We can launch this again by running the following code cell.

In [ ]:
ol.dashboard()

* _**After you have executed the above cell, you should right click the radio dashboard, and select "Create New View for Output" from the drop-down menu. This will allow you to interact with the notebook and retain access to the radio dashboard in another output view.**_

## 2. Receiving Messages
This RFSoC demonstration system has been designed to simplify data movement between the RFSoC's PS and PL. It is worth noting that the transmitter and receiver systems are entirely independant of one another. That is, they do not share common clocks and there are no hidden loopbacks in the logic fabric. To transmit and receive using the radio demonstrator, we are relying entirely on the RF DC interface.

Ascii terminals have been created using `ipywidgets` that allow you to visualise and interact with transmitted and received data. Lets start by creating the receiver terminal by running the code cell below.

In [ ]:
ol.radio_receiver.terminal()

The receiver terminal will appear. You won't be able to interact with the main text box as this terminal is for receiving messages only. That means we need to transmit a message first before it will appear in the box above. You can configure the receiver terminal using the buttons on the right. The functionality of each button is as follows:

* **Play button** - Listen for transmitted BPSK and QPSK waveforms with the extended barker sequence and print them in the terminal.
* **Stop button** - Do not listen for transmitted BPSK and QPSK waveforms with the extended barker sequence.
* **Clear button** - Clear the terminal.
* **Auto Clear button** - Automatically clear the terminal after 10 messages have been received.
* **Debug button** - When enabled, inspect the frame's meta data and payload information. When disabled, only show the payload.

Lets now put our receiver terminal to good use by transmitting a QPSK signal containing 'Hello World!'.

* _**Right-click the receiver terminal above, and in the drop-down menu that appears, select "Create New View for Output". This action will move the terminal to another window in Jupyter Labs, allowing you to scroll further down the notebook while still being able to visualise and interact with the plots.**_

## 3. Hello World!
Run the code cell below to initiate a data transfer between the RF DAC and RF ADC.

In [ ]:
ol.radio_transmitter.data('Hello World!\r')
ol.radio_transmitter.start()

Now that you have transmitted and received your first 'Hello World!' message, lets transmit another message. This time, ensure that the **Debug Button** on the receiver terminal is enabled (blue colour). You can see this button enabled in Figure 1 below. This function will allow you to inspect the frame information when you run the next code cell.

<figure> <a class="anchor" id="fig-1"></a>
    <img src="images/receiver_terminal_debug.jpg" style="width: 30%;"/>
    <figcaption><b>Figure 1: Switch on the Debug button.</b></figcaption>
</figure>

Now run the next code cell, which will transmit several QPSK frames rather than just one.

In [ ]:
ol.radio_transmitter.data(''.join(['The quick brown fox jumps over the lazy dog.\r',
                             'How razorback-jumping frogs can level six piqued gymnasts.\r']))
ol.radio_transmitter.start()

In the receiver terminal, you will now be able to see information about the packets that were just sent. A total of 3 packets will have been received. You can use this debug feature to inspect the frame meta data.

* _**From this point on you should switch off the debug feature using the debug button and clear your receiver terminal using the clear button.**_

Another terminal can be created that will allow you to insert ascii data directly into the transmitter. Running the code cell below, will create a transmitter terminal.

In [ ]:
ol.radio_transmitter.terminal()

To use this terminal, all you have to do is insert a message and press the send button. After you press the send button, the terminal will automatically clear ready for the next input. Try this now and use the receiver terminal to inspect the received data.

## 4. A Repeating Message
The next part of the demonstration is a repeating message. We will introduce a simple timer thread class, which will allow you to execute a function at a specified rate and number of iterations. Run the code cell below to create the timer thread class.

In [ ]:
import threading
import time

class TimerThread():
    def __init__(self,
                 callback,
                 rate=0.5,
                 iterations=20):
    
        self.callback = callback
        self.rate = rate
        self.iterations = iterations
        self.stopped = True
    
    def start(self):
        if self.stopped:
            thread = threading.Thread(target=self._do)
            thread.start()
            
    def _do(self):
        self.stopped = False
        iterations = 0
        while iterations < self.iterations:
            next_timer = time.time() + self.rate
            self.callback()
            iterations += 1
            sleep_time = next_timer - time.time()
            if sleep_time > 0:
                time.sleep(sleep_time)
            if self.stopped:
                break
        self.stopped = True
        
    def stop(self):
        self.stopped = True

The timer thread class accepts three arguments, a callback, the execution rate of the callback function, and the number of times the callback should be executed. We can create a callback function for the repeating message as below in `transmitter_callback()`. This function uses a global counter, to create a 'Hello World!' message with a number appended at the end. The callback will be passed to the timer thread class for execution.

Run the code cell below to create the TimerThread object and transmitter callback.

In [ ]:
counter = 0

def transmitter_callback():
    global counter
    message = ''.join(['Hello World! ', str(counter), '\r'])
    ol.radio_transmitter.data(message)
    ol.radio_transmitter.start()
    counter += 1

tx_repeater = TimerThread(callback=transmitter_callback,
                          rate=0.5,
                          iterations=20)

You can now start the thread by running the code cell below. Doing so will create a new thread that will execute the `transmitter_callback()` function 20 times, every 0.5 seconds. After the number of iterations have been achieved, the thread will exit. Before running the code cell below, ensure you can see your receiver terminal.

In [ ]:
tx_repeater.start()

If at any point you wish to stop the thread, simply use the cell below. Remember the thread will stop anyway when the number of iterations have been achieved.

In [ ]:
tx_repeater.stop()

## 5. Image Transfer
The final part of this demonstration is to transfer an image between the transmitter and receiver. We can send an image by converting it to bytes and transmitting the data for the receiver to acquire.

The code cell below performs many different tasks. Primarily, the image is prepared for transmission and displayed to the user. After each image transmission, the original image will be swapped for another that has been rotated by 90°. Rotating the image will allow us to see when a new image has been acquired at the receiver.

In [ ]:
# Import ImageViewer() from the quick widgets library
from rfsoc_radio.quick_widgets import ImageViewer
import numpy as np
import ipywidgets as ipw

# Open the target image using bytes
image = []
for i in range(4):
    file = open('images/small_cat_grey_' + str(i) + '.jpg', "rb")
    image.append(file.read())

# Set flip variable
counter = 0

# Create image viewer for transmitted image
sendimage = ImageViewer(description='Image to Transmit')

def sendimage_callback():
    global counter
    global image
    ol.radio_transmitter.data(image[counter])
    sendimage.update(image[counter])
    if counter > 2:
        counter = 0
    else:
        counter += 1
    
# Set the sendimage_callback function as the callback before transmitting
# data using the transmitter
ol.radio_transmitter.monitor.callback = [sendimage_callback]

# Slow down the transmission rate to two seconds
ol.radio_transmitter.monitor.rate = 2

sendimage.update(image[0])

# Get the widget for interaction with sent data
ipw.HBox([sendimage.get_widget()])

The image above has a resolution of 50x50 pixels and is greyscale. The code cell below will prepare the receiver for acquiring this image and displaying it to the user. Upon executing this code cell, you will see a broken image icon. Don't worry about this! As soon as an image is received, this icon will dissapear and the newly received image will take its place.

In [ ]:
# Import ImageViewer() from the quick widgets library
from rfsoc_radio.quick_widgets import ImageViewer
import numpy as np
import ipywidgets as ipw

# Create a receiver buffer to store packets of data
recvbuffer = np.empty(0, dtype=np.uint8)

# Create an image viewer object for visualising the received data
recvimage = ImageViewer(description='Received Image')

# Create a custom callback function that is executed when the receiver
# interrupt is triggered
def recvimage_callback():
    global recvbuffer
    frame = ol.radio_receiver.frame
    payload = frame["payload"]
    if ((frame["flags"] >> 1) & 1):
        recvbuffer = np.array(payload, dtype=np.uint8)
    else:
        recvbuffer = np.append(recvbuffer, payload)
    if (frame["flags"] & 1):
        recvimage.update(recvbuffer.tobytes())
        
# Set the terminal_callback function as the callback for the receiver
# when an interrupt is triggered
ol.radio_receiver.monitor.callback = [recvimage_callback]

# Get the widget for interaction with received data
ipw.HBox([recvimage.get_widget()])

The transmitter can be configured to run a callback after successfully sending data. To enable this functionality, set the transmitter to repeat and start the transfer.

In [ ]:
# Start the transfer
ol.radio_transmitter.mode = 'repeat'
ol.radio_transmitter.start()

The received image above should now contain an image and it should rotate every few seconds. You can use the radio dashboard to change the modulation scheme used by the radio system if desired. The code cell below will stop the demonstration.

In [ ]:
ol.radio_transmitter.stop()

## 3. Conclusion
This notebook has presented a simple BPSK and QPSK radio demonstrator on the RFSoC using PYNQ. It was shown that data can be transmitted and received correctly using BPSK and QPSK modulation and the RF DCs.

The next set of notebooks introduce the RFSoC's Soft-Decision Forward Error Correction (SD-FEC) block.

---

[⬅️ Previous Notebook](02_rfsoc_radio_observe.ipynb) || [Next Notebook 🚀](../notebook_H/01_fec_first_principles.ipynb)

Copyright © 2023 Strathclyde Academic Media

---
---